In [1]:
import numpy as np

from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    AutoConfig,
    TextClassificationPipeline,
)
from engine.data import prepare_data_for_fine_tuning, read_data
from typing import Any

/home/zajkoa/projects/studies/msc_2/nlp/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
MODEL_ID = "roberta-base"
MODEL_PATH = "output/2024_10_22_19_56_43/model"
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

In [3]:
test = read_data("data/split_raw/test.tsv")
test_dataset = prepare_data_for_fine_tuning(test, tokenizer)

Map: 100%|██████████| 790/790 [00:00<00:00, 5267.16 examples/s]


In [ ]:
config = AutoConfig.from_pretrained(MODEL_ID)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_PATH, config=config)
model.eval()
pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, top_k=2, device="cuda")

In [5]:
predictions = pipeline(test["text"].tolist())

In [6]:
def convert_prediction(pred: list[dict[str, Any]]) -> np.ndarray:
    if pred[0]["label"] == "LABEL_1":
        return pred[0]["score"]
    else:
        return pred[1]["score"]

In [8]:
test.label.mean()

np.float64(0.4316455696202532)

In [ ]:
((np.array(list(map(convert_prediction, predictions))) >= 0.5) == test.label.values).mean()

np.float64(0.6772151898734177)